In [153]:
import pandas as pd
import geopandas as gpd
import gtfs_functions as gtfs
import numpy as np
import pandas as pd,dateparser
from datetime import datetime, date,timedelta
from tqdm import tqdm
from shapely.geometry import LineString, Point
import osmnx  as ox
import networkx as nx
import matplotlib.pyplot as plt
import requests

In [154]:
dfSuperSet = pd.read_excel('../Varanasi Data/Transit Data/Route data/ALL ROUTE OK 24.01.2022.xlsx',sheet_name=None,header=1)

In [155]:
routes=dfSuperSet['50 Bus KM (2)']

In [156]:
routes=routes.drop(columns=['S.No'])

In [157]:
routes=routes.rename(columns={'Route No.':'route_id','Route Name':'route_long_name'})
pd.options.display.max_colwidth = 200
routes=routes.loc[0:routes.shape[0]-2]

In [158]:
routes['agency_id']=1

In [159]:
#Rename routes with two branches i.e E107 into E107A and E107 (As indicated by files name in excel)
routes['route_id'].mask((routes.route_id.shift(-1).isnull())&(routes.route_id.index!=routes.shape[0]-1), routes['route_id']+'A',inplace=True)
routes['route_id'].mask(routes.route_id.isnull(),routes['route_id'].shift().str[:-1],inplace=True)

In [161]:
#Additional features for routes.txt file
routes[['route_color', 'route_text_color']]=None
routes['route_short_name']=list(dfSuperSet.keys())[1:]
routes['route_short_name']=routes['route_short_name'].str.strip()
routes['route_long_name']=routes['route_long_name'].str.strip()

In [162]:

routes=routes[['route_id','agency_id','route_short_name','route_long_name','route_color', 'route_text_color']]


In [163]:
routes

,route_id,agency_id,route_short_name,route_long_name,route_color,route_text_color
0,E101,1,MZD-BabatpurAirport 101,Mirzamurad-Mohansarai-Cantt-LBS Airport- Babatpur Chauraha,None,None
1,E102,1,Mirzamurad-Babatpur 102,Mirzamurad-Mohansarai-Cantt-Babatpur Chauraha,None,None
2,E103,1,Mirzamurad-Cantt-Lanka103,Mirzamurad-Kachhawan-Mohansarai- Cantt- Bhelupur-Lanka,None,None
3,E104,1,Mirzamurad-Sindharo104,Mirzamurad-Mohansarai-Cantt- Bhojubeer-Sindhora,None,None
4,E105,1,Mirzamurad-Darsauna-Danganj 105,Mirzamurad-Mohansarai-Cantt-Pandaypur-Dharsauna-Danganj,None,None
5,E106,1,Mirzamurad-Chubpur-MARKAND 106,Mirzamurad-Mohansarai-Cantt-Pandeypur-Chubeypur-Markandey Mahadev,None,None
6,E107A,1,Mirzamurad-Sarnath-Lanka 107A,Mirzamurad-Mohansarai-Cantt-Ashapur-Sarnath -Lanka,None,None
7,E107,1,MZD-Lanka-Sarnath 107,Mirzamurad-Mohansarai-Cantt-Ashapur-Lanka -Sarnath,None,None
8,E108,1,MZD-DLW-Cantt-Sarnth-Lanka108,Mirzamurad- Akhari-DLW-Cantt-JHV Mall- Kachahari- Ashapur- Sarnath- Cantt-Bhelupur- Lanka-DLW- Cantt,None,None
9,E109,1,MZD-Cantt-Rajgt- DLW-Lanka109,Mirzamurad-Mohansarai-Cantt-Rajghat-Cantt-DLW-Lanka-Bhelupur-Cantt,None,None


In [45]:
routes.to_csv(r'../GTFS/routes.txt',index=False)

In [164]:
#Reads through all excel sheets and extract all stops with names and coordinates
stops=pd.DataFrame()
for route in dfSuperSet.keys():
    #skips the first sheet of excel file
    if route=='50 Bus KM (2)':
        pass
    else:
        c=pd.DataFrame(dfSuperSet[route])
        print(c.columns[0])
        #Change route name column to stops
        c=c.rename(columns={c.columns[0]:'stop_name'})
        #Checks for column that has substring Latitude in it in order to find coordinates column
        latlon=[col for col in c.columns if 'Latitude' in col]
        print(route)
        print(latlon)
        
        c=c[['stop_name',latlon[0]]]
        #Check for null rows that separates different subtables(directions)  of trips in one routes
        index_list=c[c.isnull().all(axis=1)].index
    
        stops=stops.append(c.iloc[1:index_list[0]])
        
        for i in range(len(index_list)):
            if i!= len(index_list)-1:
                stops=stops.append(c.iloc[c[c.isnull().all(axis=1)].index[i]+1:c[c.isnull().all(axis=1)].index[i+1]])
        
    
    
    

Route NO.
(E101)
MZD-BabatpurAirport 101
['Latitude\nLongitude']
Route NO.
(E102)
Mirzamurad-Babatpur 102
['Latitude\nLongitude']
Route NO.
(E103)
Mirzamurad-Cantt-Lanka103
['Latitude\nLongitude']
Route NO.
(E104)
Mirzamurad-Sindharo104
['Latitude\nLongitude']
Route No.
(E105)
Mirzamurad-Darsauna-Danganj 105
['Latitude\nLongitude']
Route No.
(E106)
Mirzamurad-Chubpur-MARKAND 106
['Latitude\nLongitude']
Route No.
(E107)
Mirzamurad-Sarnath-Lanka 107A
['Latitude\nLongitude']
Route No.
(E107)
MZD-Lanka-Sarnath 107 
['Latitude\nLongitude']
Route No.
(E108)
MZD-DLW-Cantt-Sarnth-Lanka108
['Latitude\nLongitude']
Route No.
(E109)
MZD-Cantt-Rajgt- DLW-Lanka109
['Latitude\nLongitude']
Route No.
(E110)
MZD-CANT-GODAULIA-CANTT110
['Latitude\nLongitude']
Route No.
(E111)
MZD-CANT-GODAULIA-CANTT111
['Latitude\nLongitude']


/tmp/ipykernel_2708/659472363.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stops=stops.append(c.iloc[1:index_list[0]])
/tmp/ipykernel_2708/659472363.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stops=stops.append(c.iloc[c[c.isnull().all(axis=1)].index[i]+1:c[c.isnull().all(axis=1)].index[i+1]])
/tmp/ipykernel_2708/659472363.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stops=stops.append(c.iloc[1:index_list[0]])
/tmp/ipykernel_2708/659472363.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stops=stops.append(c.iloc[c[c.isnull().all(axis=1)].index[i]+1:c[c.isnull().all(axis=1)].index[i+1]])
/tmp/ipykernel_2708/

In [165]:

stops=stops.drop_duplicates(subset=['stop_name']).reset_index(drop=True)

In [166]:
stops=stops[(stops['Latitude\nLongitude'].notnull())&(stops['Latitude\nLongitude']!='Latitude\nLongitude')].reset_index(drop=True)

In [167]:
stops

,stop_name,Latitude\nLongitude
0,MIRZAMURAD E STATION,"25.284666, 82.780556"
1,MIRZAMURAD BAZAR,"25.285235, 82.776484"
2,KIT,"25.282433, 82.791429"
3,SADHU KI KUTIYA,"25.279853, 82.799645"
4,RAKHAUNA,"25.271908, 82.821388"
...,...,...
120,TALIYABHAG,"25.327688, 82.995952"
121,MALDAHIYA,"25.324263, 82.991282"
122,KABIR CHAURA,"25.319730, 83.009175"
123,MAIDAGIN,"25.318957, 83.013142"


In [168]:
stops['stop_id']=stops.index
stops[['stop_lat','stop_lon']] = stops['Latitude\nLongitude'].str.split(", ",expand=True)
stops[['stop_lat','stop_lon']]=stops[['stop_lat','stop_lon']].astype(float)
stops.drop(columns=['Latitude\nLongitude'])

,stop_name,stop_id,stop_lat,stop_lon
0,MIRZAMURAD E STATION,0,25.284666,82.780556
1,MIRZAMURAD BAZAR,1,25.285235,82.776484
2,KIT,2,25.282433,82.791429
3,SADHU KI KUTIYA,3,25.279853,82.799645
4,RAKHAUNA,4,25.271908,82.821388
...,...,...,...,...
120,TALIYABHAG,120,25.327688,82.995952
121,MALDAHIYA,121,25.324263,82.991282
122,KABIR CHAURA,122,25.319730,83.009175
123,MAIDAGIN,123,25.318957,83.013142


In [169]:
stops[['stop_code','stop_desc','zone_id','stop_url','location_type','parent_station']]=None
stops=stops[['stop_id','stop_code','stop_name','stop_desc','stop_lat','stop_lon','zone_id','stop_url','location_type','parent_station']]

In [170]:
stops

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,0,None,MIRZAMURAD E STATION,None,25.284666,82.780556,None,None,None,None
1,1,None,MIRZAMURAD BAZAR,None,25.285235,82.776484,None,None,None,None
2,2,None,KIT,None,25.282433,82.791429,None,None,None,None
3,3,None,SADHU KI KUTIYA,None,25.279853,82.799645,None,None,None,None
4,4,None,RAKHAUNA,None,25.271908,82.821388,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
120,120,None,TALIYABHAG,None,25.327688,82.995952,None,None,None,None
121,121,None,MALDAHIYA,None,25.324263,82.991282,None,None,None,None
122,122,None,KABIR CHAURA,None,25.319730,83.009175,None,None,None,None
123,123,None,MAIDAGIN,None,25.318957,83.013142,None,None,None,None


In [53]:
stops.to_csv(r'../GTFS/stops.txt',index=False)

Creating Trips.txt File

In [184]:
#Reads through all excel sheets and extract all stops with names and coordinates
trips=pd.DataFrame(columns=['trip_id','route_id','service_id','trip_headsign','direction_id','block_id','shape_id'])
shape_id=0
for route in dfSuperSet.keys():
    #skips the first sheet of excel file
    if route=='50 Bus KM (2)':
        pass
    else:
        #Reads the whole sheet as one dataframe
        c=pd.DataFrame(dfSuperSet[route]).iloc[:-1].dropna(axis=1,how='all')
        #stores the trips from columns into a list
        tripcount=[col for col in c.columns if 'Trip' in col]
    
        index_list=c[c.isnull().all(axis=1)].index
        DirectionCount=0
        #iterates through each table of the sheet
        for i in range(len(index_list)):
            if index_list[i]==index_list[0]:
                temp=c.iloc[0:index_list[i]]
                # convert trips columns in excel sheet into rows
                for count in range(len(tripcount)):
                        loc=trips.shape[0]
                        trips.loc[loc,'block_id']=temp.loc[0,tripcount[count]]
                        trips.loc[loc,'direction_id']= "Direction " + str(DirectionCount)
                        trips.loc[loc,'shape_id']= shape_id
                #changing shape_ids and directions as we have successfully iterate over one subtable of a sheet
                shape_id+=1        
                DirectionCount= DirectionCount+1
             
            #skips consecitive empty lines   
            elif c.iloc[index_list[i-1]+1:index_list[i]].empty:
                print(i)
                print('consective nan')
                pass
            else:
                temp=c.iloc[index_list[i-1]+1:index_list[i]].reset_index(drop=True)
                
                if len(temp)!=1:
                    for count in range(len(tripcount)):
                        loc=trips.shape[0]
                        trips.loc[loc,'block_id']=temp.loc[2,tripcount[count]]
                        trips.loc[loc,'direction_id']= "Direction " + str(DirectionCount)
                        trips.loc[loc,'shape_id']= shape_id
                    shape_id+=1     
                    DirectionCount= DirectionCount+1
       
            #
            trips.route_id.mask(trips.route_id.isnull(),routes[routes['route_short_name']==route.strip()].reset_index(drop=True).route_id[0],inplace=True)   

MZD-BabatpurAirport 101
MZD-BabatpurAirport 101
Mirzamurad-Babatpur 102
Mirzamurad-Babatpur 102
Mirzamurad-Cantt-Lanka103
Mirzamurad-Cantt-Lanka103
Mirzamurad-Sindharo104
Mirzamurad-Sindharo104
Mirzamurad-Darsauna-Danganj 105
Mirzamurad-Darsauna-Danganj 105
Mirzamurad-Chubpur-MARKAND 106
1
consective nan
Mirzamurad-Chubpur-MARKAND 106
Mirzamurad-Chubpur-MARKAND 106
Mirzamurad-Chubpur-MARKAND 106
4
consective nan
Mirzamurad-Chubpur-MARKAND 106
5
consective nan
Mirzamurad-Chubpur-MARKAND 106
6
consective nan
Mirzamurad-Chubpur-MARKAND 106
7
consective nan
Mirzamurad-Chubpur-MARKAND 106
8
consective nan
Mirzamurad-Chubpur-MARKAND 106
Mirzamurad-Sarnath-Lanka 107A
Mirzamurad-Sarnath-Lanka 107A
Mirzamurad-Sarnath-Lanka 107A
MZD-Lanka-Sarnath 107 
MZD-Lanka-Sarnath 107 
MZD-Lanka-Sarnath 107 
MZD-DLW-Cantt-Sarnth-Lanka108
MZD-DLW-Cantt-Sarnth-Lanka108
MZD-DLW-Cantt-Sarnth-Lanka108
MZD-Cantt-Rajgt- DLW-Lanka109
MZD-Cantt-Rajgt- DLW-Lanka109
MZD-Cantt-Rajgt- DLW-Lanka109
MZD-CANT-GODAULIA-CANT

In [185]:
#Removes invalid trips with null block values
trips=trips[trips.block_id.notnull()].reset_index(drop=True)

In [186]:
#Currently assigned trips according to indices
trips['trip_id']=trips.index
trips['service_id']=1
trips['block_id']=trips['block_id'].str.replace('\n ','')
trips['block_id']=trips['block_id'].str.replace('\n','')
trips['block_id']=trips['block_id'].str.replace(' ','')


In [188]:
trips

,trip_id,route_id,service_id,trip_headsign,direction_id,block_id,shape_id
0,0,E101,1,NaN,Direction 0,UP65KT5041,0
1,1,E101,1,NaN,Direction 0,UP32LT1406,0
2,2,E101,1,NaN,Direction 0,UP65LT1409,0
3,3,E101,1,NaN,Direction 0,UP32PN6056,0
4,4,E101,1,NaN,Direction 0,UP65KT5041,0
...,...,...,...,...,...,...,...
257,257,E111,1,NaN,Direction 2,UP65LT1395,32
258,258,E111,1,NaN,Direction 2,UP32PN8724,32
259,259,E111,1,NaN,Direction 2,UP32KT5047,32
260,260,E111,1,NaN,Direction 2,UP65KT5042,32


In [189]:
trips.to_csv(r'../GTFS/trips.txt',index=False)

Creating Stop_times.txt File

In [190]:
#Reads through all excel sheets and extract all stops with names and coordinates
stop_times=pd.DataFrame(columns=['trip_id',	'arrival_time',	'departure_time',	'stop_sequence',	'stop_head_sign',	'shape_dist_traveled',	'halt',	'stop_name'])
trip_id=0
for route in dfSuperSet.keys():
    #skips the first sheet of excel file
    if route=='50 Bus KM (2)':
        pass
    else:
        c=pd.DataFrame(dfSuperSet[route]).iloc[:-1].dropna(axis=1,how='all')

        #stores the trips from columns into a list
        tripcount=[col for col in c.columns if 'Trip' in col]
        #had to hard code some of the sheets because it did not follow general patterns
        if route=='MZD-Cantt-Rajgt- DLW-Lanka109':
            tripcount=tripcount[2:]
              
        #We dont have shape_dist_travel in shape file for all possible point but since it has a feature in raw data so I included that stop_times
        total_km=[col for col in c.columns if 'Km' in col][0]
        index_list=c[c.isnull().all(axis=1)].index
        if route=='Mirzamurad-Chubpur-MARKAND 106':
            index_list=index_list[:3]

        #iterates through each table of the sheet
        for i in range(len(index_list)):
            if index_list[i]==index_list[0]:
                temp=c.iloc[1:index_list[i]]
                temp=temp.rename(columns={c.columns[0]:'stop_name'})
                for count in range(len(tripcount)):

                        stop_times=stop_times.append({'stop_name':temp['stop_name'].to_list(),'departure_time':temp[tripcount[count]].to_list(),'halt':temp['HLT'].to_list(),'shape_dist_traveled':temp[total_km].to_list()},ignore_index=True)
                        stop_times.trip_id.mask(stop_times.trip_id.isnull(),trip_id,inplace=True)
                        stop_times=stop_times.explode(['stop_name','departure_time','halt','shape_dist_traveled'])
                        trip_id=trip_id+1
                        
                        
                
                
            #skips consecitive empty lines   
            elif c.iloc[index_list[i-1]+1:index_list[i]].empty:
                print(i)
                print('consective nan')
                pass
            else:
                
                temp=c.iloc[index_list[i-1]+4:index_list[i]].reset_index(drop=True)
                temp=temp.rename(columns={c.columns[0]:'stop_name'})
                temp=temp[temp['stop_name'].notnull()]
                
                if len(temp)!=1:
                    for count in range(len(tripcount)):

                        stop_times=stop_times.append({'stop_name':temp['stop_name'].to_list(),'departure_time':temp[tripcount[count]].to_list(),'halt':temp['HLT'].to_list(),'shape_dist_traveled':temp[total_km].to_list()},ignore_index=True)
                        stop_times.trip_id.mask(stop_times.trip_id.isnull(),trip_id,inplace=True)
                        stop_times=stop_times.explode(['stop_name','departure_time','halt','shape_dist_traveled'])
                        trip_id=trip_id+1 

/tmp/ipykernel_2708/733403283.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stop_times=stop_times.append({'stop_name':temp['stop_name'].to_list(),'departure_time':temp[tripcount[count]].to_list(),'halt':temp['HLT'].to_list(),'shape_dist_traveled':temp[total_km].to_list()},ignore_index=True)
/tmp/ipykernel_2708/733403283.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stop_times=stop_times.append({'stop_name':temp['stop_name'].to_list(),'departure_time':temp[tripcount[count]].to_list(),'halt':temp['HLT'].to_list(),'shape_dist_traveled':temp[total_km].to_list()},ignore_index=True)
/tmp/ipykernel_2708/733403283.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stop_times=stop_times.append({'stop_name':temp['stop

1
consective nan


/tmp/ipykernel_2708/733403283.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stop_times=stop_times.append({'stop_name':temp['stop_name'].to_list(),'departure_time':temp[tripcount[count]].to_list(),'halt':temp['HLT'].to_list(),'shape_dist_traveled':temp[total_km].to_list()},ignore_index=True)
/tmp/ipykernel_2708/733403283.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stop_times=stop_times.append({'stop_name':temp['stop_name'].to_list(),'departure_time':temp[tripcount[count]].to_list(),'halt':temp['HLT'].to_list(),'shape_dist_traveled':temp[total_km].to_list()},ignore_index=True)
/tmp/ipykernel_2708/733403283.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stop_times=stop_times.append({'stop_name':temp['stop

In [191]:
stop_times=stop_times.reset_index(drop=True)


In [192]:
#Merging stop_times with stops on stop_name to get stop_ids
stop_times=stop_times.merge(stops[['stop_id','stop_name']], how='left',on='stop_name')

In [193]:
stop_times['departure_time']=pd.to_datetime(stop_times['departure_time'], format='%H%M%S', errors='ignore')

In [194]:
#Generating stop sequences for the all trips
sequence=0
for i, row in stop_times.iterrows():
    stop_times.loc[i,'arrival_time']= (datetime.combine(date(1,1,1),stop_times.loc[i,'departure_time'])- timedelta(minutes=stop_times.loc[i,'halt'])).time()
    if i==0:
        stop_times.loc[i,'stop_sequence']=sequence
        sequence=sequence+1
    elif stop_times.loc[i,'trip_id']==stop_times.loc[i-1,'trip_id']:
        stop_times.loc[i,'stop_sequence']=sequence
        sequence=sequence+1
        
    else:
        stop_times.loc[i,'stop_sequence']=0
        sequence=1
    


In [195]:
stop_times['stop_id']=stop_times['stop_id'].astype('Int64')
stop_times=stop_times[['trip_id','stop_id','arrival_time','departure_time','stop_sequence','stop_head_sign','shape_dist_traveled']]

In [196]:
stop_times

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_head_sign,shape_dist_traveled
0,0,0,05:50:00,05:50:00,0,NaN,0
1,0,1,05:52:00,05:57:00,1,NaN,1
2,0,2,06:01:00,06:01:00,2,NaN,3
3,0,3,06:03:00,06:03:00,3,NaN,4
4,0,4,06:07:00,06:07:00,4,NaN,6
...,...,...,...,...,...,...,...
7449,261,4,22:46:00,22:46:00,52,NaN,60
7450,261,3,22:50:00,22:50:00,53,NaN,62
7451,261,2,22:52:00,22:52:00,54,NaN,63
7452,261,1,22:56:00,22:56:00,55,NaN,65


In [66]:
stop_times.to_csv(r'../GTFS/stop_times.txt',index=False)

In [93]:
#Generates a list of geographical coordinates representing linestring from origin stop to destination stop
def shape(stop_coord,next_stop_coord):
    x = requests.get("https://api.mapbox.com/directions/v5/mapbox/driving/"+str(stop_coord[1])+"%2C"+str(stop_coord[0])+"%3B"+str(next_stop_coord[1])+"%2C"+str(next_stop_coord[0])+"?alternatives=true&geometries=geojson&language=en&overview=simplified&steps=true&access_token=pk.eyJ1IjoienVscWFyYSIsImEiOiJjbGVkNWxmMDMwMzN1M3lzMmF6bGkzZm9zIn0.sabyhYfG-Oj-juPhfyWhrg")
    return x.json()['routes'][0]['geometry']['coordinates']



In [150]:
#Reads through all excel sheets and extract all stops with names and coordinates
shapes=pd.DataFrame(columns=['shape_id','shape_pt_lat','shape_pt_lon','shape_pt_sequence'])
shape_id=0
for route in dfSuperSet.keys():
    #skips the first sheet of excel file
    if route=='50 Bus KM (2)':
        pass
    else:
        print(route)
        c=pd.DataFrame(dfSuperSet[route]).iloc[:-1].dropna(axis=1,how='all')
        c=c.rename(columns={c.columns[0]:'stop_name'})
        #gets the column name for coordinates
        latlon=[col for col in c.columns if 'Latitude' in col][0]
        
              
        
        #Keeps track of null rows between subtables
        index_list=c[c.isnull().all(axis=1)].index
        if route=='Mirzamurad-Chubpur-MARKAND 106':
            index_list=index_list[:3]
        
        #iterates through each table of the sheet
        for i in range(len(index_list)):
            #checks the first table of the sheet
            if index_list[i]==index_list[0]:
                temp=c.iloc[1:index_list[i]]
                temp['next_stop_coord']=temp[latlon].shift(-1)
                temp['next_stop']=temp['stop_name'].shift(-1)
                temp=temp[temp['next_stop_coord'].notnull()]
                
                
                shape_point_sequence=0
                #creates a temporary table for one shape
                temp_shapes=pd.DataFrame(columns=['shape_id','shape_pt_lat','shape_pt_lon','shape_pt_sequence'])
                #iterates through all stops visited in one shape and provides list of points between them
                for k,v in temp.iterrows():
                    #Get coordinates for the origin and destination stops
                        stop_coord = [float(n) for n in v[latlon].split(",")]
                        next_stop_coord=[float(n) for n in v['next_stop_coord'].split(",")]

                        shape_list_of_geometry = shape(stop_coord, next_stop_coord)
                        temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
                 
                #create rows for each geographical coordinate visited between stops        
                temp_shapes=temp_shapes.explode('shape_pt_lat').reset_index(drop=True)
                temp_shapes[['shape_pt_lon','shape_pt_lat']]=pd.DataFrame(temp_shapes['shape_pt_lat'].tolist(), index= temp_shapes.index)
                
                temp_shapes['shape_id']=shape_id
                temp_shapes['shape_pt_sequence']=temp_shapes.index
                #appends this table into main shape table
                shapes=shapes.append(temp_shapes).reset_index(drop=True)        
                shape_id+=1
                
                   
                
                
            #skips consecitive empty lines   
            elif c.iloc[index_list[i-1]+1:index_list[i]].empty:
                print(i)
                print('consective nan')
                pass
            else:
                #checks the later tables of the sheet
                temp=c.iloc[index_list[i-1]+4:index_list[i]].reset_index(drop=True)
                #works same as first table
                temp['next_stop_coord']=temp[latlon].shift(-1)
                temp['next_stop']=temp['stop_name'].shift(-1)
                temp=temp[temp['next_stop_coord'].notnull()]

                shape_point_sequence=0
                temp_shapes=pd.DataFrame(columns=['shape_id','shape_pt_lat','shape_pt_lon','shape_pt_sequence'])
                for k,v in temp.iterrows():
                        stop_coord = [float(n) for n in v[latlon].split(",")]
                        next_stop_coord=[float(n) for n in v['next_stop_coord'].split(",")]

                        shape_list_of_geometry = shape(stop_coord, next_stop_coord)
                        temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
                        
                temp_shapes=temp_shapes.explode('shape_pt_lat').reset_index(drop=True)
                temp_shapes[['shape_pt_lon','shape_pt_lat']]=pd.DataFrame(temp_shapes['shape_pt_lat'].tolist(), index= temp_shapes.index)
                temp_shapes['shape_id']=shape_id
                temp_shapes['shape_pt_sequence']=temp_shapes.index
                shapes=shapes.append(temp_shapes).reset_index(drop=True)        
                shape_id+=1
        

MZD-BabatpurAirport 101
MZD-BabatpurAirport 101


/tmp/ipykernel_2708/1908612718.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['next_stop_coord']=temp[latlon].shift(-1)
/tmp/ipykernel_2708/1908612718.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['next_stop']=temp['stop_name'].shift(-1)
/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore

Mirzamurad-Babatpur 102
Mirzamurad-Babatpur 102


/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'sh

Mirzamurad-Cantt-Lanka103
Mirzamurad-Cantt-Lanka103


...

/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'sh

MZD-CANT-GODAULIA-CANTT110
MZD-CANT-GODAULIA-CANTT110


/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'sh

MZD-CANT-GODAULIA-CANTT111
MZD-CANT-GODAULIA-CANTT111


/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'shape_pt_lat':shape_list_of_geometry}, ignore_index=True)
/tmp/ipykernel_2708/1908612718.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_shapes=temp_shapes.append({'sh

In [152]:
shapes

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,0,25.284410,82.780492,0
1,0,25.282874,82.788947,1
2,0,25.281990,82.792961,2
3,0,25.283655,82.793669,3
4,0,25.283546,82.794604,4
...,...,...,...,...
8272,32,25.285850,82.776031,529
8273,32,25.285274,82.775819,530
8274,32,25.285149,82.776464,531
8275,32,25.285149,82.776464,532


In [ ]:
shapes.to_csv(r'../GTFS/shapes.txt',index=False)